# 原生爬虫
* 明确目的
* 找到数据对应的网页
* 分析网页的结构，以及数据所在的标签位置

## 使用库
* Beautiful Soup 正则表达式
* Scrapy 爬虫框架

## 反爬虫
* ip --> 代理ip库

## 模拟服务器请求
获取到服务器返回给我们的HTML
用正则表达式提取我们要的数据(名字、人气) 

## 模块代码测试

In [1]:
from urllib import request
from io import BytesIO
import gzip

r = request.urlopen('https://www.douyu.com/g_LOL')
htmls = r.read()
buff = BytesIO(htmls)
f = gzip.GzipFile(fileobj=buff)
htmls = f.read().decode('utf-8')
# 这里是用gzip压缩过的代码，需要解码
htmls

'<!-- 分区详情 --><!-- 分区定制化 --><!doctype html><!--[if lte IE 7]><html class="ie ie7 acie"><![endif]--><!--[if IE 8]><html class="ie ie8"><![endif]--><!--[if IE 9]><html class="ie ie9"><![endif]--><html><head><meta charset="UTF-8"><title>英雄联盟直播_lol直播_英雄联盟直播视频解说攻略_斗鱼直播</title><meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1"><meta name="renderer" content="webkit"><meta name="keywords" content="英雄联盟,lol,英雄联盟直播,lol直播,英雄联盟直播间,英雄联盟直播地址,网游竞技直播,英雄联盟视频解说攻略,热门游戏直播,高清游戏直播,电子竞技直播,lol大神直播"><meta name="description" content="斗鱼英雄联盟直播专区拥有海量的英雄联盟lol直播内容,热门英雄联盟赛事直播解说和精彩lol英雄联盟直播内容全天候不间断,英雄联盟名玩大神主播芜湖大司马、东北大鹌鹑、洞主凯哥在这里带你各种操作秀翻天,一起嗨翻全场,全新lol英雄联盟直播尽在斗鱼直播。"><meta http-equiv="Content-Security-Policy" content="\n    default-src \'self\';\n    img-src * data:;\n    media-src * blob:;\n    font-src *;\n    frame-src *;\n    worker-src * blob:;\n    object-src *;\n    connect-src *;\n    script-src \'self\' \'unsafe-inline\' \'unsafe-eval\' *.qq.com *.geetest.com *.vod.lxdns.com *.videojj.com *.douyu.com 

In [2]:
# 正则表达式测试

In [3]:
import re
root_pattern = '<div class="DyListCover-info">[\s\S]*?</div>'
root_htmls = re.findall(root_pattern, htmls)
root_htmls = list(root_htmls)[1::2]
print(root_htmls)

['<div class="DyListCover-info"><span class="DyListCover-hot is-template"><svg><use xlink:href="#icon-hot_8a57f0b"></use></svg>511.2万</span><h2 class="DyListCover-user is-template"><svg><use xlink:href="#icon-user_c95acf8"></use></svg>即将拥有人鱼线的PDD</h2></div>', '<div class="DyListCover-info"><span class="DyListCover-hot is-template"><svg><use xlink:href="#icon-hot_8a57f0b"></use></svg>23.7万</span><h2 class="DyListCover-user is-template"><svg><use xlink:href="#icon-user_c95acf8"></use></svg>咸鱼光y</h2></div>', '<div class="DyListCover-info"><span class="DyListCover-hot is-template"><svg><use xlink:href="#icon-hot_8a57f0b"></use></svg>235万</span><h2 class="DyListCover-user is-template"><svg><use xlink:href="#icon-user_c95acf8"></use></svg>东北大鹌鹑</h2></div>', '<div class="DyListCover-info"><span class="DyListCover-hot is-template"><svg><use xlink:href="#icon-hot_8a57f0b"></use></svg>207.3万</span><h2 class="DyListCover-user is-template"><svg><use xlink:href="#icon-user_c95acf8"></use></svg>芜湖大司

In [4]:
'''
'''
from urllib import request
from io import BytesIO
import gzip
import re

class Spider():
    url = 'https://www.douyu.com/g_LOL'
    
    # 注意里面原生html中为双引号
    # 正则表达式：匹配中间所有内容，非贪婪模式,自己体会，给需要匹配的内容增加括号成为组
    root_pattern = '<div class="DyListCover-info">([\s\S]*?)</div>'
    name_pattern1 = '</svg>([\s\S]*?)</h2>'
    name_pattern2 = '</svg>(.*)'
    number_pattern = '</svg>([\s\S]*?)</span>'
    
    def __fetch_content(self):
        r = request.urlopen(Spider.url)
        
        # 转码
        htmls = r.read()
        
        # 由于经过压缩，所以需要解压
        buff = BytesIO(htmls)
        f = gzip.GzipFile(fileobj=buff)
        htmls = f.read().decode('utf-8')
        return htmls
    
    def __analysis(self,htmls):
        root_htmls = re.findall(Spider.root_pattern, htmls)
        root_htmls = list(root_htmls)[1::2]
        anchors = []
        for html in root_htmls:
            name = re.findall(Spider.name_pattern1,html)
            
            # 注意这里正则表达式匹配以后返回的值类型是list
            name = re.findall(Spider.name_pattern2,name[0])
            number = re.findall(Spider.number_pattern,html)
            anchor = {'name':name,'number':number}
            anchors.append(anchor)
        return anchors
    
    def __refine(self,anchors):
        '''
        格式化匹配内容：去除无用字符
        string.strip() 去除前后空白字符串的内置函数
        '''
        l = lambda anchor:{
            'name':anchor['name'][0].strip(),
            'number':anchor['number'][0]
        }
        return map(l,anchors)
        
    def __sort(self,anchors):
        # key传入函数需要注意
        anchors = sorted(anchors,key=self.__sort_seed,reverse=True)
        return anchors
    
    def __sort_seed(self,anchor):
        r = re.findall('\d*',anchor['number'])
        number = float(r[0])
        if '万' in anchor['number']:
            number *= 10000
        return number
    
    def __show(self,anchors):
        for rank in range(0,len(anchors)):
            print('rank'+str(rank+1)
                 + ':'+anchors[rank]['name']
                 + ' '+anchors[rank]['number'])
    
    # 总控函数
    def go(self): 
        htmls = self.__fetch_content()
        anchors = self.__analysis(htmls)
        anchors = list(self.__refine(anchors))
        anchors = self.__sort(anchors)
        self.__show(anchors)

spider = Spider()
spider.go()

rank1:即将拥有人鱼线的PDD 511.2万
rank2:东北大鹌鹑 235万
rank3:芜湖大司马丶 207.3万
rank4:南波儿大魔王丶 192.4万
rank5:不2不叫周淑怡 166.4万
rank6:小苏菲 132.2万
rank7:骆歆冲冲冲 118.6万
rank8:山东棋王弈哥 79.7万
rank9:JDG丶Zoom 78.3万
rank10:JDG丶Kanavi 77.3万
rank11:impOUO 68.5万
rank12:叶音符 68.3万
rank13:放逐大帝灬 65.9万
rank14:草莓 63.2万
rank15:JDG丶LokeN 61.1万
rank16:狐狸酱大魔王 50.3万
rank17:刘小又XD 49.3万
rank18:OMGH4cker 45万
rank19:安妮今天五杀了吗 44.8万
rank20:魔獸后裔 42万
rank21:电棍 41.3万
rank22:狗康Bfate丶 41万
rank23:杰克螳螂 40.6万
rank24:黑白锐雯 40.3万
rank25:远古解说YD 39.4万
rank26:英雄联盟经典赛事点播 35.4万
rank27:峡谷杀神酱油瓶 32.3万
rank28:国服第一冰鸟黑马 31.4万
rank29:OMGCurse 31.1万
rank30:初恋女孩水水 31.1万
rank31:ESTAR丶Xiaobai 31.1万
rank32:格局太小OoO 31万
rank33:英雄联盟赛事 30.9万
rank34:ESTAR丶ShiauC 30.1万
rank35:Mouse光酱 28.2万
rank36:冷爷行妹 26.6万
rank37:熊王c 25.4万
rank38:LOL丶摇摆 24.9万
rank39:披萨解说 24万
rank40:咸鱼光y 23.7万
rank41:她打了个嗝儿 22.3万
rank42:小酒窝翁馨 22.2万
rank43:作作金克丝 21.5万
rank44:睿智三 20.8万
rank45:JDG丶Peace 20.2万
rank46:猫咪老师ADKing 18.4万
rank47:岁月刀妹QAQ 18.3万
rank48:小明剑魔 18.2万
rank49:风中追风Fzz1 18.1万
rank50:77小茉莉 18万


爬取框架节点：
document.querySelector("#listAll > div.layout-Module-container.layout-Cover.ListContent > ul > li:nth-child(1) > div > a.DyListCover-wrap > div.DyListCover-content > div:nth-child(1)")
```html
<div class="DyListCover-info">
   <span class="DyListCover-hot"><svg class="DyListCover-hotIcon"><use xlink:href="#icon-hot_8a57f0b"></use></svg>39.3万</span>
    <h2 class="DyListCover-user"><svg class="DyListCover-userIcon"><use xlink:href="#icon-user_c95acf8"></use></svg>峡谷杀神酱油瓶</h2>
</div>
```